In [54]:
import torch
import torch.nn as nn
import sys
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
import pandas as pd
from torch.nn import functional as F


In [61]:
files = Path('../../datasets/balanced/2019/VIEGAS/XX').iterdir()

df = pd.DataFrame()

for file in files:
    temp = pd.read_csv(file)
    df = pd.concat([df, temp])

df_labels = df[['class']]

df = df.drop(columns=['MAWILAB_taxonomy', 'MAWILAB_distance', 'MAWILAB_nbDetectors', 'MAWILAB_label', 'class'])

dataset = torch.tensor(df.to_numpy()).float()
labels = torch.tensor(df_labels.to_numpy().reshape(-1)).double()

In [64]:
df['Shit'] = 0
df

,VIEGAS_countPackets,VIEGAS_countBytes,VIEGAS_averagePacketSize,VIEGAS_percentagePushed,VIEGAS_percentageSynFin,VIEGAS_percentageFin,VIEGAS_percentageSyn,VIEGAS_percentageAck,VIEGAS_percentageRst,VIEGAS_percentageICMPRedirect,...,VIEGAS_percentageSynFin_A,VIEGAS_percentageFin_A,VIEGAS_percentageSyn_A,VIEGAS_percentageAck_A,VIEGAS_percentageRst_A,VIEGAS_percentageICMPRedirect_A,VIEGAS_percentageICMPTimeExceeded_A,VIEGAS_percentageICMPUnreacheable_A,VIEGAS_percentageICMPOtherTypes_A,Shit
0,1,60,60.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,1.0,0
1,2,120,60.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,1.0,0
2,1,60,60.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,1.0,0
3,1,54,54.00,0.00,0.0,0.0,1.00,0.0,0.0,0.0,...,0.0,0.00,0.99,0.00,0.01,0.0,0.0,0.0,1.0,0
4,1,54,54.00,0.00,0.0,0.0,1.00,0.0,0.0,0.0,...,0.0,0.00,1.00,0.00,0.00,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,1,54,54.00,0.00,0.0,0.0,1.00,0.0,0.0,0.0,...,0.0,0.00,0.99,0.00,0.01,0.0,0.0,0.0,1.0,0
12814,1,54,54.00,0.00,0.0,0.0,1.00,0.0,0.0,0.0,...,0.0,0.00,1.00,0.00,0.00,0.0,0.0,0.0,1.0,0
12815,1,54,54.00,0.00,0.0,0.0,1.00,0.0,0.0,0.0,...,0.0,0.00,1.00,0.00,0.00,0.0,0.0,0.0,1.0,0
12816,11,5090,462.73,0.45,0.0,0.0,0.09,1.0,0.0,0.0,...,0.0,0.01,0.06,0.94,0.00,0.0,0.0,0.0,1.0,0


In [49]:
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.model=nn.Sequential(
            nn.Flatten(),
            nn.Linear(48, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        z = self.model(x)
        return F.log_softmax(z, dim=1)

In [41]:
class CustomMawiDataset(Dataset):
    def __init__(self, year='2019', month='XX'):
        files = Path(f'../../datasets/balanced/{year}/VIEGAS/{month}').iterdir()

        df = pd.DataFrame()

        for file in files:
            temp = pd.read_csv(file)
            df = pd.concat([df, temp])

        self.df_labels = df[['class']]

        self.df = df.drop(columns=['MAWILAB_taxonomy', 'MAWILAB_distance', 'MAWILAB_nbDetectors', 'MAWILAB_label', 'class'])

        self.dataset = torch.tensor(self.df.to_numpy()).float()
        self.labels = torch.tensor(self.df_labels.to_numpy().reshape(-1)).long()
        
        print(self.dataset.shape)
        print(self.labels.shape)
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx], self.labels[idx]

In [42]:
mawi = CustomMawiDataset()

torch.Size([12818, 48])
torch.Size([12818])


In [46]:
dl = DataLoader(mawi, batch_size=32, shuffle=True)

In [60]:
model = SimpleModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion=nn.CrossEntropyLoss()

for i in range(5):
    for b, (X_train, y_train) in enumerate(dl):
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, 1)[1]
        batch_cor = (predicted == y_train).sum()

        print(f'Loss: {loss.item():.4f} - Correct: {100 * batch_cor / len(y_pred):.2f}%')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
        
        

Loss: 947.2841 - Correct: 37.50%
Loss: 454.8363 - Correct: 50.00%
Loss: 269.3509 - Correct: 43.75%
Loss: 737.6278 - Correct: 53.12%
Loss: 342.3047 - Correct: 65.62%
Loss: 422.0831 - Correct: 56.25%
Loss: 265.7004 - Correct: 53.12%
Loss: 104.0857 - Correct: 53.12%
Loss: 58.4051 - Correct: 75.00%
Loss: 156.6463 - Correct: 71.88%
Loss: 89.9815 - Correct: 59.38%
Loss: 86.3592 - Correct: 59.38%
Loss: 117.8264 - Correct: 62.50%
Loss: 209.0813 - Correct: 56.25%
Loss: 47.5089 - Correct: 59.38%
Loss: 95.4875 - Correct: 50.00%
Loss: 26.5545 - Correct: 59.38%
Loss: 63.7157 - Correct: 71.88%
Loss: 24.4010 - Correct: 50.00%
Loss: 169.0930 - Correct: 50.00%
Loss: 15.1008 - Correct: 65.62%
Loss: 16.3931 - Correct: 56.25%
Loss: 91.5307 - Correct: 56.25%
Loss: 172.3006 - Correct: 40.62%
Loss: 17.8004 - Correct: 62.50%
Loss: 146.2734 - Correct: 46.88%
Loss: 105.3697 - Correct: 59.38%
Loss: 169.1441 - Correct: 50.00%
Loss: 56.8470 - Correct: 68.75%
Loss: 71.5766 - Correct: 65.62%
Loss: 178.5425 - Correct

Loss: 0.6465 - Correct: 62.50%
Loss: 0.6794 - Correct: 62.50%
Loss: 0.6975 - Correct: 40.62%
Loss: 0.6755 - Correct: 53.12%
Loss: 0.6766 - Correct: 59.38%
Loss: 0.6758 - Correct: 62.50%
Loss: 0.6640 - Correct: 65.62%
Loss: 0.6694 - Correct: 56.25%
Loss: 0.6769 - Correct: 46.88%
Loss: 0.6870 - Correct: 59.38%
Loss: 0.6903 - Correct: 56.25%
Loss: 0.6591 - Correct: 43.75%
Loss: 0.6808 - Correct: 53.12%
Loss: 0.6396 - Correct: 53.12%
Loss: 0.6794 - Correct: 65.62%
Loss: 0.7037 - Correct: 40.62%
Loss: 0.6853 - Correct: 59.38%
Loss: 0.6642 - Correct: 40.62%
Loss: 0.6912 - Correct: 53.12%
Loss: 0.6610 - Correct: 46.88%
Loss: 0.6680 - Correct: 56.25%
Loss: 0.6811 - Correct: 43.75%
Loss: 0.6686 - Correct: 59.38%
Loss: 0.6946 - Correct: 50.00%
Loss: 0.6817 - Correct: 62.50%
Loss: 0.6692 - Correct: 53.12%
Loss: 0.7014 - Correct: 37.50%
Loss: 0.6837 - Correct: 43.75%
Loss: 0.6711 - Correct: 53.12%
Loss: 0.7171 - Correct: 46.88%
Loss: 0.6711 - Correct: 53.12%
Loss: 0.6823 - Correct: 40.62%
Loss: 0.

Loss: 0.6919 - Correct: 56.25%
Loss: 0.6954 - Correct: 50.00%
Loss: 0.7494 - Correct: 34.38%
Loss: 0.6936 - Correct: 46.88%
Loss: 0.6718 - Correct: 50.00%
Loss: 0.6498 - Correct: 53.12%
Loss: 0.6876 - Correct: 53.12%
Loss: 0.7403 - Correct: 43.75%
Loss: 0.6944 - Correct: 37.50%
Loss: 0.7185 - Correct: 31.25%
Loss: 0.6755 - Correct: 40.62%
Loss: 0.6970 - Correct: 46.88%
Loss: 0.6657 - Correct: 50.00%
Loss: 0.6601 - Correct: 56.25%
Loss: 0.6930 - Correct: 53.12%
Loss: 0.6529 - Correct: 50.00%
Loss: 0.6688 - Correct: 71.88%
Loss: 0.7091 - Correct: 59.38%
Loss: 1.2063 - Correct: 40.62%
Loss: 0.6691 - Correct: 59.38%
Loss: 0.7028 - Correct: 25.00%
Loss: 0.6699 - Correct: 56.25%
Loss: 0.6973 - Correct: 53.12%
Loss: 0.8222 - Correct: 65.62%
Loss: 0.6920 - Correct: 53.12%
Loss: 0.6671 - Correct: 62.50%
Loss: 0.7065 - Correct: 46.88%
Loss: 0.6827 - Correct: 37.50%
Loss: 0.6850 - Correct: 50.00%
Loss: 0.6969 - Correct: 43.75%
Loss: 0.6869 - Correct: 62.50%
Loss: 0.6991 - Correct: 46.88%
Loss: 0.

Loss: 0.6947 - Correct: 46.88%
Loss: 0.6724 - Correct: 50.00%
Loss: 0.6934 - Correct: 50.00%
Loss: 0.6934 - Correct: 50.00%
Loss: 0.6879 - Correct: 62.50%
Loss: 0.6920 - Correct: 53.12%
Loss: 0.6919 - Correct: 53.12%
Loss: 0.6695 - Correct: 56.25%
Loss: 0.6727 - Correct: 50.00%
Loss: 0.7023 - Correct: 34.38%
Loss: 0.6659 - Correct: 62.50%
Loss: 0.6744 - Correct: 46.88%
Loss: 0.7003 - Correct: 37.50%
Loss: 0.6887 - Correct: 59.38%
Loss: 0.6741 - Correct: 46.88%
Loss: 0.6905 - Correct: 56.25%
Loss: 0.6934 - Correct: 50.00%
Loss: 0.6920 - Correct: 53.12%
Loss: 0.6934 - Correct: 50.00%
Loss: 0.6947 - Correct: 46.88%
Loss: 0.7286 - Correct: 40.62%
Loss: 0.6981 - Correct: 37.50%
Loss: 0.6892 - Correct: 62.50%
Loss: 0.6952 - Correct: 43.75%
Loss: 0.6949 - Correct: 43.75%
Loss: 0.6960 - Correct: 37.50%
Loss: 0.6699 - Correct: 62.50%
Loss: 0.6943 - Correct: 40.62%
Loss: 0.6930 - Correct: 53.12%
Loss: 0.6929 - Correct: 65.62%
Loss: 0.6936 - Correct: 43.75%
Loss: 0.6931 - Correct: 50.00%
Loss: 0.

Loss: 0.6996 - Correct: 43.75%
Loss: 0.7000 - Correct: 43.75%
Loss: 0.7031 - Correct: 40.62%
Loss: 0.6943 - Correct: 50.00%
Loss: 0.6942 - Correct: 50.00%
Loss: 0.6885 - Correct: 56.25%
Loss: 0.6827 - Correct: 62.50%
Loss: 0.6826 - Correct: 62.50%
Loss: 0.6863 - Correct: 37.50%
Loss: 0.6761 - Correct: 68.75%
Loss: 0.6881 - Correct: 56.25%
Loss: 0.6809 - Correct: 43.75%
Loss: 0.6912 - Correct: 53.12%
Loss: 0.7012 - Correct: 43.75%
Loss: 0.6879 - Correct: 56.25%
Loss: 0.6945 - Correct: 50.00%
Loss: 0.6978 - Correct: 46.88%
Loss: 0.7010 - Correct: 43.75%
Loss: 0.6975 - Correct: 46.88%
Loss: 0.6973 - Correct: 46.88%
Loss: 0.6799 - Correct: 65.62%
Loss: 0.6998 - Correct: 43.75%
Loss: 0.6860 - Correct: 59.38%
Loss: 0.7047 - Correct: 37.50%
Loss: 0.6685 - Correct: 56.25%
Loss: 0.6915 - Correct: 53.12%
Loss: 0.6939 - Correct: 50.00%
Loss: 0.6915 - Correct: 53.12%
Loss: 0.6872 - Correct: 59.38%
Loss: 0.6916 - Correct: 53.12%
Loss: 0.7002 - Correct: 40.62%
Loss: 0.6979 - Correct: 43.75%
Loss: 0.

Loss: 0.7273 - Correct: 50.00%
Loss: 0.6932 - Correct: 50.00%
Loss: 0.6911 - Correct: 65.62%
Loss: 0.6922 - Correct: 56.25%
Loss: 0.6945 - Correct: 43.75%
Loss: 0.6932 - Correct: 50.00%
Loss: 0.6909 - Correct: 59.38%
Loss: 0.6960 - Correct: 40.62%
Loss: 0.6914 - Correct: 56.25%
Loss: 0.6953 - Correct: 43.75%
Loss: 0.6922 - Correct: 53.12%
Loss: 0.6986 - Correct: 34.38%
Loss: 0.6952 - Correct: 43.75%
Loss: 0.6932 - Correct: 50.00%
Loss: 0.6947 - Correct: 43.75%
Loss: 0.6926 - Correct: 53.12%
Loss: 0.6951 - Correct: 37.50%
Loss: 0.6935 - Correct: 46.88%
Loss: 0.6928 - Correct: 59.38%
Loss: 0.6932 - Correct: 40.62%
Loss: 0.6929 - Correct: 65.62%
Loss: 0.6935 - Correct: 43.75%
Loss: 0.6924 - Correct: 59.38%
Loss: 0.6912 - Correct: 65.62%
Loss: 0.6932 - Correct: 50.00%
Loss: 0.6973 - Correct: 34.38%
Loss: 0.6915 - Correct: 56.25%
Loss: 0.6903 - Correct: 59.38%
Loss: 0.6944 - Correct: 46.88%
Loss: 0.6933 - Correct: 50.00%
Loss: 0.6934 - Correct: 50.00%
Loss: 0.6920 - Correct: 53.12%
Loss: 0.

Loss: 0.6951 - Correct: 46.88%
Loss: 0.6854 - Correct: 65.62%
Loss: 0.6968 - Correct: 43.75%
Loss: 0.6968 - Correct: 43.75%
Loss: 0.6919 - Correct: 53.12%
Loss: 0.6951 - Correct: 46.88%
Loss: 0.6919 - Correct: 53.12%
Loss: 0.6949 - Correct: 46.88%
Loss: 0.6906 - Correct: 56.25%
Loss: 0.7004 - Correct: 34.38%
Loss: 0.6959 - Correct: 43.75%
Loss: 0.6944 - Correct: 46.88%
Loss: 0.6914 - Correct: 56.25%
Loss: 0.6893 - Correct: 65.62%
Loss: 0.6963 - Correct: 37.50%
Loss: 0.6953 - Correct: 40.62%
Loss: 0.6958 - Correct: 34.38%
Loss: 0.6934 - Correct: 46.88%
Loss: 0.6932 - Correct: 46.88%
Loss: 0.6925 - Correct: 59.38%
Loss: 0.6942 - Correct: 43.75%
Loss: 0.6925 - Correct: 53.12%
Loss: 0.6924 - Correct: 53.12%
Loss: 0.6911 - Correct: 56.25%
Loss: 0.6921 - Correct: 53.12%
Loss: 0.6889 - Correct: 59.38%
Loss: 0.6918 - Correct: 53.12%
Loss: 0.6956 - Correct: 46.88%
Loss: 0.6916 - Correct: 53.12%
Loss: 0.6962 - Correct: 46.88%
Loss: 0.6939 - Correct: 50.00%
Loss: 0.6889 - Correct: 56.25%
Loss: 0.